# Notebook requêtant wikidata en SPARQL

Imports

In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON
import numpy as np
import pandas as pd
import folium
from folium.plugins import MarkerCluster

### 1) Récupération de tous les rois de France

In [2]:
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
sparql.setQuery("""
SELECT ?item ?itemLabel (GROUP_CONCAT(DISTINCT ?spouseLabel;separator=", ") AS ?spouses) (GROUP_CONCAT(DISTINCT ?mannerOfDeathLabel;separator=", ") AS ?mannersOfDeath) 
?dateOfBirth ?placeOfBirthLabel ?dateOfDeath ?placeOfDeathLabel ?placeOfBurialLabel ?fatherLabel ?motherLabel ?conflictLabel ?startTime ?endTime

WHERE
{
{?item wdt:P31 wd:Q5 .} 
{?item wdt:P39 wd:Q18384454 .} UNION {?item wdt:P39 wd:Q22923081} UNION {?item wdt:P39 wd:Q3439798}
{?item wdt:P569 ?dateOfBirth} UNION {MINUS {?item wdt:P569 ?dateOfBirth}}
{?item wdt:P19 ?placeOfBirth} UNION {MINUS {?item wdt:P19 ?placeOfBirth}}
{?item wdt:P570 ?dateOfDeath} UNION {MINUS {?item wdt:P570 ?dateOfDeath}}
{?item wdt:P19 ?placeOfDeath} UNION {MINUS {?item wdt:P19 ?placeOfDeath}}
{?item wdt:P509 ?mannerOfDeath} UNION {MINUS {?item wdt:P509 ?mannerOfDeath}}
{?item wdt:P119 ?placeOfBurial} UNION {MINUS {?item wdt:P119 ?placeOfBurial}}
{?item wdt:P22 ?father} UNION {MINUS {?item wdt:P22 ?father}}
{?item wdt:P25 ?mother} UNION {MINUS {?item wdt:P25 ?mother}}
{?item wdt:P26 ?spouse} UNION {MINUS {?item wdt:P26 ?spouse}}
{?item wdt:P607 ?conflict} UNION {MINUS {?item wdt:P607 ?conflict}}
?item p:P39 ?positionHeld.
{?positionHeld pq:P580 ?startTime.} UNION {MINUS {?positionHeld pq:P580 ?startTime.}}
{?positionHeld pq:P582 ?endTime.} UNION {MINUS {?positionHeld pq:P582 ?endTime.}}
  
SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],fr" .
                         ?item rdfs:label ?itemLabel .
                         ?placeOfBirth rdfs:label ?placeOfBirthLabel .
                         ?placeOfDeath rdfs:label ?placeOfDeathLabel .
                         ?mannerOfDeath rdfs:label ?mannerOfDeathLabel .
                         ?placeOfBurial rdfs:label ?placeOfBurialLabel .
                         ?father rdfs:label ?fatherLabel .
                         ?mother rdfs:label ?motherLabel .
                         ?spouse rdfs:label ?spouseLabel .}
}GROUP BY ?item ?itemLabel ?dateOfBirth ?placeOfBirthLabel ?dateOfDeath ?placeOfDeathLabel ?placeOfBurialLabel ?fatherLabel ?motherLabel ?conflictLabel ?startTime ?endTime

""")

sparql.setReturnFormat(JSON)
results = sparql.query().convert()

In [3]:
results

{'head': {'vars': ['item',
   'itemLabel',
   'spouses',
   'mannersOfDeath',
   'dateOfBirth',
   'placeOfBirthLabel',
   'dateOfDeath',
   'placeOfDeathLabel',
   'placeOfBurialLabel',
   'fatherLabel',
   'motherLabel',
   'conflictLabel',
   'startTime',
   'endTime']},
 'results': {'bindings': [{'item': {'type': 'uri',
     'value': 'http://www.wikidata.org/entity/Q134259'},
    'itemLabel': {'xml:lang': 'fr',
     'type': 'literal',
     'value': 'Louis VII de France'},
    'dateOfBirth': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime',
     'type': 'literal',
     'value': '1120-01-01T00:00:00Z'},
    'placeOfBirthLabel': {'xml:lang': 'fr',
     'type': 'literal',
     'value': 'Paris'},
    'dateOfDeath': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime',
     'type': 'literal',
     'value': '1180-09-25T00:00:00Z'},
    'placeOfDeathLabel': {'xml:lang': 'fr',
     'type': 'literal',
     'value': 'Paris'},
    'placeOfBurialLabel': {'xml:lang': 'fr',
     'type

In [4]:
rois = []
for r in results['results']['bindings']:
    roi = {}
    if("itemLabel" in r):
        roi['nom'] = r['itemLabel']['value']
    if("dateOfBirth" in r):
        roi['dateOfBirth'] = r['dateOfBirth']['value']
    if("placeOfBirthLabel" in r):
        roi['placeOfBirthLabel'] = r['placeOfBirthLabel']['value']
    if("dateOfDeath" in r):
        roi['dateOfDeath'] = r['dateOfDeath']['value']
    if("placeOfDeathLabel" in r):
        roi['placeOfDeathLabel'] = r['placeOfDeathLabel']['value']
    if("mannerOfDeathLabel" in r):
        roi['mannerOfDeathLabel'] = r['mannerOfDeathLabel']['value']
    if("placeOfBurialLabel" in r):
        roi['placeOfBurialLabel'] = r['placeOfBurialLabel']['value']
    if("fatherLabel" in r):
        roi['fatherLabel'] = r['fatherLabel']['value']
    if("motherLabel" in r):
        roi['motherLabel'] = r['motherLabel']['value']
    if("spouses" in r):
        roi['spouses'] = r['spouses']['value']
    if("conflictLabel" in r):
        roi['conflictLabel'] = r['conflictLabel']['value']
    if("startTime" in r):
        roi['startTime'] = r['startTime']['value']
    if("endTime" in r):
        roi['endTime'] = r['endTime']['value']
    rois.append(roi)

In [14]:
df = pd.DataFrame(rois)
df.count()

nom                   170
dateOfBirth           167
placeOfBirthLabel     149
dateOfDeath           166
placeOfDeathLabel     149
placeOfBurialLabel    110
fatherLabel           154
motherLabel           136
spouses               170
startTime              80
endTime                80
dtype: int64

In [15]:
df = df.drop_duplicates()

In [16]:
df.count()

nom                   170
dateOfBirth           167
placeOfBirthLabel     149
dateOfDeath           166
placeOfDeathLabel     149
placeOfBurialLabel    110
fatherLabel           154
motherLabel           136
spouses               170
startTime              80
endTime                80
dtype: int64

In [17]:
pd.options.display.max_rows = 999

In [19]:
df = df.drop_duplicates(subset=["nom"])

In [21]:
df.count()

nom                   120
dateOfBirth           117
placeOfBirthLabel     102
dateOfDeath           116
placeOfDeathLabel     102
placeOfBurialLabel     67
fatherLabel           106
motherLabel            89
spouses               120
startTime              50
endTime                50
dtype: int64

In [ ]:
df.sort_values(by=['dateOfBirth'])

### 4) Save in csv

In [ ]:
df.to_csv("rois-france.csv")